# CLIXO Tree builder

Cleanup tree data created with Cytoscape and link anntation from mapping file.

In [30]:
import json
import pandas as pd


# Load mapping

with open('./data/clixo-mapping.json', 'r') as f:
    clixo_map = json.load(f)
    
print(len(clixo_map.keys()))

1811


In [32]:
with open('./data/clixo-tree-layout.cyjs', 'r') as f:
    clixo = json.load(f)

nodes = clixo['elements']['nodes']
len(nodes)

4805

In [34]:
print(nodes[1])

{'selected': False, 'position': {'x': 15220.0, 'y': 25564.0}, 'data': {'shared_name': 'CLIXO:8037', 'name': 'CLIXO:8037', 'SUID': 24102, 'selected': False, 'geneCount': 3.0, 'id_original': 'CLIXO:8037', 'id': '24102'}}


In [33]:
from goatools import obo_parser

# Annotation file for the CLIXO terms
oboUrl = './data/go.obo'

obo = obo_parser.GODag(oboUrl, optional_attrs=['def'])

load obo file ./data/go.obo
./data/go.obo: fmt(1.2) rel(2017-01-13) 47,943 GO Terms


In [37]:
new_nodes = []

id2clixo = {}
gcounts = []

for n in nodes:
    d = n['data']
    
    id2clixo[d['id']] = d['id_original']
    clixo_id = d['id_original'].split(':')[1]
    name = d['id_original']
    genes = int(d['geneCount'])
    
    gcounts.append(genes)
    
    if clixo_id in clixo_map.keys():
        goid = clixo_map[clixo_id]['go']
        name = obo[goid].name
        
    data = {
        'id': d['id_original'],
        'name': name,
        'geneCount': genes
    }
    
    new_nodes.append({
            'data': data,
            'position': n['position']
        })

In [38]:
len(nodes)

4805

In [39]:
max(gcounts)

5886

In [40]:
# Cleanup edges

edges = clixo['elements']['edges']
new_edges = []

for e in edges:
    d = e['data']
    
    data = {
        'source': id2clixo[d['source']],
        'target': id2clixo[d['target']],
    }
    
    new_edges.append({'data': data})

In [41]:
clixo_compact = {
    'data': {
        'name': 'CLIXO Tree'
    },
    'elements': {
        'nodes': new_nodes,
        'edges': new_edges
    }
}

with open('./data/clixo-compact.json', 'w') as outfile:
    json.dump(clixo_compact, outfile)

## And same for GO


In [ ]:
!wget 'https://gist.githubusercontent.com/keiono/b7c047c1166681ef7170881217819938/raw/b81de1da63075ceb0e4c8d99adeacf83d0193cdb/goWOgenes.cyjs' -O ./data/go-final2.cyjs




In [ ]:
with open('./data/go-final2.cyjs', 'r') as f:
    gotree = json.load(f)

In [ ]:
import pandas as pd

df = pd.read_csv('./data/collapsed_go.no_IGI.propagated.term_sizes', sep='\t', names=['id', 'genes'])
df.head(10)

In [ ]:
math.exp(max(df['genes'])/1000)

In [ ]:
go2genes = {}

for row in df.itertuples():
    go2genes[row[1]] = row[2].item()

In [ ]:
nodes = gotree['elements']['nodes']
nodes[1]

In [ ]:
id2go = {}
new_nodes = []
import math

for n in nodes:
    d = n['data']
    
    id2go[d['id']] = d['id_original']
    name = d['name']
    genes = go2genes[d['id_original']]
        
    data = {
        'id': d['id_original'],
        'name': name,
        'geneCount': genes,
        'size': math.exp(genes/1000),
        'namespace': d['namespace']
    }
    
    new_nodes.append({
            'data': data,
            'position': n['position']
        })

In [ ]:
new_nodes[1]

In [ ]:
math.exp(6000/1000)

In [ ]:
edges = gotree['elements']['edges']
new_edges = []


for e in edges:
    d = e['data']
    
    data = {
        'source': id2go[d['source']],
        'target': id2go[d['target']],
        'branch': d['branch']
    }
    
    new_edges.append({'data': data})

In [ ]:
new_edges[1]

In [ ]:
go_compact = {
    'data': {
        'name': 'GO Tree'
    },
    'elements': {
        'nodes': new_nodes,
        'edges': new_edges
    }
}

with open('./data/go-compact.json', 'w') as outfile:
    json.dump(go_compact, outfile)